<a href="https://colab.research.google.com/github/alivia0921/Fake-New-Detection-Text-Sentiment-Analysis/blob/main/CSSItextSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

dataset: https://www.kaggle.com/datasets/jruvika/fake-news-detection

In [ ]:
#importing necesary libraries
import nltk
import spacy
import re
import pandas as pd
import numpy as np
import gensim
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn import metrics
from matplotlib import pyplot
import itertools
from sklearn.neighbors import KNeighborsClassifier
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Dense

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
#importing dataset
dpath = "/content/data.csv"
df = pd.read_csv(dpath)

df.head()

FileNotFoundError: ignored

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
#combining the headline and body into one column
df['Body'] = df['Body'].fillna(' ')
df['CompleteNews'] = df['Headline'] + " " + df['Body']
df.isnull().sum()

In [ ]:
df['Label'].value_counts()

In [ ]:
df.head()

In [ ]:
#dropping old columns
df = df.drop(["URLs", "Headline", "Body"], axis = 1)

In [ ]:
df.head()

In [ ]:
#checking to see if the data is balanced
sns.countplot(df['Label'])
plt.show()

In [ ]:
#removing words that have little to no significance in the meaning of phrases - such as 'the', 'of', etc.
nltk.download('stopwords')
stop_word = set(stopwords.words('english'))
print(stop_word)

In [ ]:
#only taking the alphabet, not punction, and converts it to lowercase
ps = PorterStemmer()
def textProcess (news):
  news = news.lower()
  news = re.sub('[^a-zA-Z]',' ', news)  #only take alphabets
  news = news.split()
  helper = []
  for i in news:
    if i in stop_word:
      pass
    else:
      helper.append(ps.stem(i))

  news = ' '.join(helper)
  return news



In [ ]:
textProcess(df['CompleteNews'][0])
# test the preprocessing method

In [ ]:
#now saving completed news at the processed text
newsProcessed = [textProcess(i) for i in tqdm(df['CompleteNews'])]
df['CompleteNews'] = newsProcessed
df.head()

In [ ]:
#saving the real news in a seperate list
dfTrue = df[df['Label']==1].reset_index()
TrueNews = []
for i in range(1872):
  TrueNews.append(dfTrue['CompleteNews'][i])

TrueNews = ' '.join(TrueNews) #join the array


In [ ]:
#saving the fake news in a seperate list
dfFake = df[df['Label']==0].reset_index()
FakeNews = []
for i in range(2137):
  FakeNews.append(dfFake['CompleteNews'][i])

FakeNews = ' '.join(FakeNews) #join the array

In [ ]:
#creating a wordcloud for real news
wordcloud = WordCloud(collocations = False,
                background_color ='white',
                min_font_size = 10).generate(TrueNews)

# plot the WordCloud image
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

In [ ]:
#creating a wordcloud for fake news
wordcloud = WordCloud(collocations = False,
                background_color ='white',
                min_font_size = 10).generate(FakeNews)

# plot the WordCloud image
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

# Using Bag of Words

In [ ]:
#creates a vector based on the frequency of each word in the text
# processedNewsInput = CountVectorizer(ngram_range=(1,2), max_df=0.9, min_df=3, max_features=2400).fit_transform(newsProcessed).toarray()

In [ ]:
processedNewsInput = CountVectorizer(ngram_range=(1,2), max_df=0.9, min_df=3, max_features=2400).fit_transform(newsProcessed).toarray()


In [ ]:
target = df['Label']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(processedNewsInput, target, test_size=0.3, random_state=0)

In [ ]:
def print_4_metrics(target_test, predicted):
  print("%-12s %f" % ('Accuracy:', metrics.accuracy_score(target_test,predicted)))
  print("%-12s %f" % ('Precision:', metrics.precision_score(target_test, predicted,labels=None, pos_label=1, average='binary', sample_weight=None)))
  print("%-12s %f" % ('Recall:', metrics.recall_score(target_test, predicted,labels=None, pos_label=1, average='binary', sample_weight=None)))
  print("%-12s %f" % ('F1 Score:', metrics.f1_score(target_test, predicted,labels=None, pos_label=1, average='binary', sample_weight=None)))

In [ ]:
# Helper function that allows you to draw nicely formatted confusion matrices
def draw_confusion_matrix(y, yhat, classes):
    '''
        Draws a confusion matrix for the given target and predictions
        Adapted from scikit-learn and discussion example.
    '''
    plt.cla()
    plt.clf()
    matrix = confusion_matrix(y, yhat)
    plt.imshow(matrix, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.colorbar()
    num_classes = len(classes)
    plt.xticks(np.arange(num_classes), classes, rotation=90)
    plt.yticks(np.arange(num_classes), classes)

    fmt = 'd'
    thresh = matrix.max() / 2.
    for i, j in itertools.product(range(matrix.shape[0]), range(matrix.shape[1])):
        plt.text(j, i, format(matrix[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if matrix[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

In [ ]:
accList = []
precisionList = []
recallList = []
F1List = []

Logistic Regression, l1 regularization

In [ ]:
#prepares and fits a logistic regression model using L1 regularization
log_reg = LogisticRegression(penalty = 'l1', solver = 'liblinear')
log_reg.fit(x_train, y_train)

In [ ]:
log_predicted = log_reg.predict(x_test)
log_score = log_reg.predict_proba(x_test)[:, 1]

print_4_metrics(y_test, log_predicted)

fpr_log_reg, tpr_log_reg, thresholds = metrics.roc_curve(y_test,log_score)
print("Logistic Model Performance Results:\n")
pyplot.figure(1)
pyplot.plot(fpr_log_reg, tpr_log_reg, color='orange', lw=1)
pyplot.title("ROC curve with Logistic Regression")
pyplot.xlabel('FPR')
pyplot.ylabel('TPR')

aucroc = metrics.auc(fpr_log_reg, tpr_log_reg)
print('AUC of ROC: ', aucroc)

accList.append (metrics.accuracy_score(y_test, log_predicted))
precisionList.append (metrics.precision_score(y_test, log_predicted))
recallList.append(metrics.recall_score(y_test, log_predicted))
F1List.append(metrics.f1_score(y_test, log_predicted))

In [ ]:
draw_confusion_matrix(log_predicted, y_test, ['Fake', 'True'])

In [ ]:
#runs svm
svm = SVC(probability = True)

svm.fit(x_train, y_train)

predicted = svm.predict(x_test)

SVM_score = svm.predict_proba(x_test)[:,1]

accList.append (metrics.accuracy_score(y_test, predicted))
precisionList.append (metrics.precision_score(y_test, predicted))
recallList.append(metrics.recall_score(y_test, predicted))
F1List.append(metrics.f1_score(y_test, predicted))

In [ ]:
#runs decision tree
DTC = DecisionTreeClassifier()
DTC.fit(x_train, y_train)
predicted = DTC.predict(x_test)
accList.append (metrics.accuracy_score(y_test, predicted))
precisionList.append (metrics.precision_score(y_test, predicted))
recallList.append(metrics.recall_score(y_test, predicted))
F1List.append(metrics.f1_score(y_test, predicted))

In [ ]:
#runs random forest
RF = RandomForestClassifier()
RF.fit(x_train, y_train)
predicted = RF.predict(x_test)
accList.append (metrics.accuracy_score(y_test, predicted))
precisionList.append (metrics.precision_score(y_test, predicted))
recallList.append(metrics.recall_score(y_test, predicted))
F1List.append(metrics.f1_score(y_test, predicted))

In [ ]:
#runs multinomial naive bayes
MNB = MultinomialNB()
MNB.fit(x_train, y_train)
predicted = MNB.predict(x_test)
accList.append (metrics.accuracy_score(y_test, predicted))
precisionList.append (metrics.precision_score(y_test, predicted))
recallList.append(metrics.recall_score(y_test, predicted))
F1List.append(metrics.f1_score(y_test, predicted))

In [ ]:
#finding the best value for k
k_values = [1,2,3,5,7,9,10,20,200]

for k in k_values:
    # run knn here:
    # note: when create KNN object, assign parameter "n_neighbors" to k
    # referece: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
    '''your code goes below here'''
    knnClassifier = KNeighborsClassifier(n_neighbors=k)
    knnClassifier.fit(x_train, y_train)
    predicted = knnClassifier.predict(x_test)

    # report the accuracy here:
    # note: print out k value and use metrics.accuracy_score() do show accuracy
    '''your code goes below here'''
    print("k value: ", k)
    print("Accuracy score: ", metrics.accuracy_score(y_test, predicted))



In [ ]:
#runs kNN
knnClassifier = KNeighborsClassifier(n_neighbors=1)
knnClassifier.fit(x_train, y_train)
predicted = knnClassifier.predict(x_test)
accList.append (metrics.accuracy_score(y_test, predicted))
precisionList.append (metrics.precision_score(y_test, predicted))
recallList.append(metrics.recall_score(y_test, predicted))
F1List.append(metrics.f1_score(y_test, predicted))

In [ ]:
models = ['Logistic Regression','SVM','Decision Tree','Random Forest','Multinomial Naive Bayes', 'KNN']


In [ ]:
cv_models_accuracy = pd.DataFrame({'Models':models, 'Accuracy':accList, 'Precision':precisionList, 'Recall': recallList, 'F1': F1List})
cv_models_accuracy

# Using TFIDF

In [ ]:
#creates an array with the weighted tfidf values for each word instead of the basic one previously used
tdv = TfidfVectorizer(ngram_range=(1,2), max_df=0.9, min_df=3, max_features=2400)
TFIDFVec = tdv.fit_transform(newsProcessed).toarray()

In [ ]:
target = df['Label']
x_train, x_test, y_train, y_test = train_test_split(TFIDFVec, target, test_size=0.3, random_state=0)

In [ ]:
accList2= []
precisionList2 = []
recallList2 = []
F1List2 = []

In [ ]:
#simply runs all the models again using this new array
log_reg = LogisticRegression(penalty = 'l1', solver = 'liblinear')
log_reg.fit(x_train, y_train)
log_predicted = log_reg.predict(x_test)
accList2.append (metrics.accuracy_score(y_test, log_predicted))
precisionList2.append (metrics.precision_score(y_test, log_predicted))
recallList2.append(metrics.recall_score(y_test, log_predicted))
F1List2.append(metrics.f1_score(y_test, log_predicted))

In [ ]:
svm = SVC(probability = True)

svm.fit(x_train, y_train)

predicted = svm.predict(x_test)

SVM_score = svm.predict_proba(x_test)[:,1]

accList2.append (metrics.accuracy_score(y_test, predicted))
precisionList2.append (metrics.precision_score(y_test, predicted))
recallList2.append(metrics.recall_score(y_test, predicted))
F1List2.append(metrics.f1_score(y_test, predicted))

In [ ]:
def fakeDetect(news):
  input = [news]
  processedNewsInput = tdv.transform(input).toarray()
  predicted = svm.predict(processedNewsInput)
  if (predicted == [0]):
    print("Fake news")
  elif (predicted == [1]):
    print("Real news")

In [ ]:
fakeDetect('the new york times selects gilber cruz as its next books editor')
fakeDetect('warning something big is about to happen in amnerica the martial law is coming -- 100% chance it will happen')

In [ ]:
DTC = DecisionTreeClassifier()
DTC.fit(x_train, y_train)
predicted = DTC.predict(x_test)
accList2.append (metrics.accuracy_score(y_test, predicted))
precisionList2.append (metrics.precision_score(y_test, predicted))
recallList2.append(metrics.recall_score(y_test, predicted))
F1List2.append(metrics.f1_score(y_test, predicted))


In [ ]:
RF = RandomForestClassifier()
RF.fit(x_train, y_train)
predicted = RF.predict(x_test)
accList2.append (metrics.accuracy_score(y_test, predicted))
precisionList2.append (metrics.precision_score(y_test, predicted))
recallList2.append(metrics.recall_score(y_test, predicted))
F1List2.append(metrics.f1_score(y_test, predicted))

In [ ]:
MNB = MultinomialNB()
MNB.fit(x_train, y_train)
predicted = MNB.predict(x_test)
accList2.append (metrics.accuracy_score(y_test, predicted))
precisionList2.append (metrics.precision_score(y_test, predicted))
recallList2.append(metrics.recall_score(y_test, predicted))
F1List2.append(metrics.f1_score(y_test, predicted))

In [ ]:
knnClassifier = KNeighborsClassifier(n_neighbors=1)
knnClassifier.fit(x_train, y_train)
predicted = knnClassifier.predict(x_test)
accList2.append (metrics.accuracy_score(y_test, predicted))
precisionList2.append (metrics.precision_score(y_test, predicted))
recallList2.append(metrics.recall_score(y_test, predicted))
F1List2.append(metrics.f1_score(y_test, predicted))

In [ ]:
models = ['Logistic Regression','SVM','Decision Tree','Random Forest','Multinomial Naive Bayes', 'KNN']

In [ ]:
cv_models_accuracy2 = pd.DataFrame({'Models':models, 'Accuracy':accList2, 'Precision':precisionList2, 'Recall': recallList2, 'F1': F1List2})
cv_models_accuracy2

# Using WordToVec

In [ ]:
#list of words
corpus = [sentence.split() for sentence in newsProcessed]

In [ ]:
#model that converts to vectors
WordToVec = gensim.models.Word2Vec(corpus, window=5, min_count=2)

In [ ]:
def AvgWordToVec(text):
    return np.mean([WordToVec.wv[i] for i in text if i in WordToVec.wv.vocab], axis = 0)

In [ ]:
#actually converting the data to vectors
vectorised = []
for doc in tqdm(corpus):
    vectorised.append(AvgWordToVec(doc))

In [ ]:
#converting the vectors into an array
word2VecInput = np.array(vectorised)
target = df['Label']
x_train, x_test, y_train, y_test = train_test_split(word2VecInput, target, random_state=0, test_size=0.3)

In [ ]:
accList3= []
precisionList3 = []
recallList3 = []
F1List3 = []

In [ ]:
#once again, just running the same models with this new array
log_reg = LogisticRegression(penalty = 'l1', solver = 'liblinear')
log_reg.fit(x_train, y_train)
log_predicted = log_reg.predict(x_test)
accList3.append (metrics.accuracy_score(y_test, log_predicted))
precisionList3.append (metrics.precision_score(y_test, log_predicted))
recallList3.append(metrics.recall_score(y_test, log_predicted))
F1List3.append(metrics.f1_score(y_test, log_predicted))

In [ ]:
svm = SVC(probability = True)

svm.fit(x_train, y_train)

predicted = svm.predict(x_test)

SVM_score = svm.predict_proba(x_test)[:,1]

accList3.append (metrics.accuracy_score(y_test, predicted))
precisionList3.append (metrics.precision_score(y_test, predicted))
recallList3.append(metrics.recall_score(y_test, predicted))
F1List3.append(metrics.f1_score(y_test, predicted))

In [ ]:
DTC = DecisionTreeClassifier()
DTC.fit(x_train, y_train)
predicted = DTC.predict(x_test)
accList3.append (metrics.accuracy_score(y_test, predicted))
precisionList3.append (metrics.precision_score(y_test, predicted))
recallList3.append(metrics.recall_score(y_test, predicted))
F1List3.append(metrics.f1_score(y_test, predicted))

In [ ]:
RF = RandomForestClassifier()
RF.fit(x_train, y_train)
predicted = RF.predict(x_test)
accList3.append (metrics.accuracy_score(y_test, predicted))
precisionList3.append (metrics.precision_score(y_test, predicted))
recallList3.append(metrics.recall_score(y_test, predicted))
F1List3.append(metrics.f1_score(y_test, predicted))

In [ ]:
knnClassifier = KNeighborsClassifier(n_neighbors=1)
knnClassifier.fit(x_train, y_train)
predicted = knnClassifier.predict(x_test)
accList3.append (metrics.accuracy_score(y_test, predicted))
precisionList3.append (metrics.precision_score(y_test, predicted))
recallList3.append(metrics.recall_score(y_test, predicted))
F1List3.append(metrics.f1_score(y_test, predicted))

In [ ]:
models = ['Logistic Regression','SVM','Decision Tree','Random Forest', 'KNN']

In [ ]:
cv_models_accuracy3 = pd.DataFrame({'Models':models, 'Accuracy':accList3, 'Precision':precisionList3, 'Recall': recallList3, 'F1': F1List3})
cv_models_accuracy3

# LSTM

In [ ]:
def WordToVecCombined(text, length=500):
    sent_vec = np.array([WordToVec.wv[i] for i in text if i in WordToVec.wv.vocab])
    if(sent_vec.shape[0] > length):
      sent_vec = sent_vec[:length]

    sent_vec_new = np.zeros((length, sent_vec.shape[1]))
    sent_vec_new[:sent_vec.shape[0]] = sent_vec
    return sent_vec_new

In [ ]:
#
corpus_new = np.array([WordToVecCombined(sent) for sent in tqdm(corpus)])

In [ ]:
corpus_new[0]

In [ ]:
#creates an rNN using LSTM
model = Sequential()
model.add(LSTM(150))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

In [ ]:
model(corpus_new)

In [ ]:
target = df['Label']
x_train, x_test, y_train, y_test = train_test_split(corpus_new, target, random_state=0, test_size=0.3)

In [ ]:
#trains and goes through the model 10 times
history = model.fit(x_train,y_train,epochs = 10)

In [ ]:
pred_prob = model(x_test)
pred = np.where(pred_prob>=0.5, 1, 0)

In [ ]:
#uses a confusion matrix to represent the predictions
cm = confusion_matrix(y_test, pred)
plt.figure(figsize=(5,5))
sns.heatmap(cm, annot=True, fmt='', cbar=False, linewidths=2,
            xticklabels = ['Fake','Real'], yticklabels = ['Fake','Real'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted', color='black', fontsize=15)
plt.ylabel('Actual', color='black', fontsize=15);

In [ ]:
#uses one hot encoding for all the words
oneHotEncoded = [one_hot(words, 5000) for words in df['CompleteNews']]

In [ ]:
#stores the length of words in each news article
len_list = []
for w in df['CompleteNews']:
    w = w.split()
    len_list.append(len(w))

print('Summary of word length :')
pd.Series(len_list).describe()

In [ ]:
# taking sentences length as 400 based on the info above
sent_length = 360
# cuts the longer ones to 400 and adds more to the ones with less
embedded_text = pad_sequences(oneHotEncoded, padding='pre', maxlen=sent_length)
embedded_text[:5]

In [ ]:
'''
practically doing the same thing as done in the previous model
this time embeddes the words and converts them at the same time as performing LSTM
instead of using word2vec beforehand and feeding those vectors
'''
model = Sequential()
model.add(Embedding(5000, 100, input_length=sent_length))
model.add(LSTM(150))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
# splitting the dataset into train and test
x_train, x_test, y_train, y_test = train_test_split(np.array(embedded_text), df.Label, test_size=0.3, random_state=3)

In [ ]:
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.33, random_state=3)

In [ ]:
x_test.shape, x_val.shape, x_train.shape

In [ ]:
# fitting the model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=64)

In [ ]:
#another confusion matrix except this time it's much more accurate
pred_prob = model(x_test)
pred = np.where(pred_prob>=0.5, 1, 0)

cm = confusion_matrix(y_test, pred)
plt.figure(figsize=(5,5))
sns.heatmap(cm, annot=True, fmt='', cbar=False, linewidths=2,
            xticklabels = ['Fake','Real'], yticklabels = ['Fake','Real'])
plt.title('confusion matrix')
plt.xlabel('Predicted', color='black', fontsize=15)
plt.ylabel('Actual', color='black', fontsize=15);